### 支持向量机 Support Vector Machines, SVM

#### 优点和缺点
> 优点：泛化错误率低，计算开销不大，结果易解释   
> 缺点：对参数调节和核函数选择敏感，原始分类器不加修改仅适用于处理二类问题

- 支持向量机的特性为：1. 支持向量机的超平面构建问题与动态规划类似，都是将大的优化问题分割成多个小优化问题逐步逼近最优结果；2. 在分割后的多个小问题优化的过程中，超平面上的每个点都是由且仅由其支持向量决定的，此时甚至删去非支持向量也不影响这次问题的优化结果，这也是支持向量机的名字的由来。

#### - ___**这里的实例关注SVM最流行的实现，即序列最小优化(SMO)和选择核函数(Kernel)将SVM拓展到更多数据集上**____

### 一些概念的解释：

#### - 线性可分:
> - 以二维空间的二类分类为例，假设该二维空间上的点(二类)可以被一条直线将两个分类分割，则称该组数据线性可分。
#### - 分隔超平面(separating hyperplane):
> - 依旧以上述的二维空间二类分类为例，将两种数据分隔的直线称为分隔超平面；这个例子可以推广到更高维度的空间，三维空间中的分隔超平面为平面，四维及更高维的空间中的分隔超平面就叫超平面。(分隔)超平面就是分类的决策边界，分布在超平面一侧的所有数据都被分到一类，如果数据点距离该决策边界越远，则该数据的分类结果就越可信。
#### - 间隔(margin)：
> - 找到最优的分隔超平面，我们希望找到离分隔超平面最近的点，并确保这些点距离分隔超平面尽可能远，这个点到面的距离就是间隔

### 寻找最大间隔：

#### - 求解数据集的最佳分隔直线：
> - 以上述的分类问题为例，示例数据集的分隔超平面的形式可以写成$\mathbf{w}^T\mathbf{x}+b$，要计算点A到分隔超平面的距离，就必须给出点到分隔面的法线或者垂线长度，该值为$\frac{\lvert \mathbf{w}^T\mathbf{A}+b \rvert}{\lvert \lvert \mathbf{w} \rvert \rvert}$，此处的常数$b$类似于Logistic回归中的截距$w_0$。这里的向量$\mathbf{w}$和常数$b$一起描述了所给数据的分隔线或者超平面。

#### - 分类器求解的优化问题：

> - 分类器的工作原理：输入数据->输出数据类别标签，在二类分类问题下，这个作用过程和Unit Step函数很相似，为了可以用一个函数描述两个标签，我们定义标签类型为1和-1。  
> - 当计算数据点到分隔面的距离并确定分隔面的放置位置时，间隔通过$label*(\mathbf{w}^T\mathbf{x}+b)$(函数间隔)来衡量，此时，若点在超平面(假设)正向侧，则label为1，反之为-1，而距离超平面的间隔可以用该公式统一计算出来。而目标就是找到分类器中定义的$\mathbf{w}$和$b$，为此需要找到数据中具有最小间隔的数据点，也就是支持向量，然后就要对支持向量间隔最大化，可以写作：$argmax_{\mathbf{w},b}[min_{(label*(\mathbf{w}^T\mathbf{x}+b))}*\frac{1}{\lvert \lvert \mathbf{w} \rvert \rvert}]$(几何间隔)，也就是最优化支持向量的间隔，优化乘积的一个常见方法就是固定一个乘子，然后优化另一个乘子，此例固定了$label*(\mathbf{w}^T\mathbf{x}+b)$，将其固定为1(基乘子)，此时只需要优化$\frac{1}{\lvert\lvert\mathbf{w}\rvert\rvert}$，于是这个问题就变成了带有约束条件的优化问题，约束条件就是$label*(\mathbf{w}^T\mathbf{x}+b)\geq1.0$，在这里我们引入 ___**拉格朗日乘子法**___ 来求解：
> - 通过引入拉格朗日乘子，就可以基于约束条件表述问题，此例中数据为离散的数据点，于是可以以基于数据点的形式、引入拉格朗日乘子的方式来描述优化目标函数：
$max_{\alpha}[\sum_{i=1}^{m}\alpha-\frac{1}{2}\sum_{i,j=1}^{m}label^i \cdot label^j \cdot \alpha_i \cdot \alpha_j \cdot \langle x^i, x^j \rangle]$
其约束条件为：
$\alpha \geq 0, \sum_{i-1}^{m}\alpha_i \cdot label^i = 0$
但是以上表述都基于一个假设：___**数据必须100%线性可分**___，即上述运算表述的是将数据集完全分隔的硬间隔(hard margin)最大化问题。但是很明显数据100%线性可分是相当理想的，于是可以引入 ___松弛变量(slack variable)___ 来允许部分数据点可以位于超平面错误的一侧，这种间隔称为软间隔(soft margin)，这时的约束条件为：
$C \geq \alpha \geq 0, \sum_{i-1}^{m}\alpha_i \cdot label^i = 0$
其中$C$为常数(调参=_=，又是TMD调参)，用于**控制最大化间隔**和**保证大部分点的函数间隔小于1.0**这两个目标的权重，因此可以通过调节$C$得到不同的结果，一旦求出了所有的$\alpha$，分隔超平面可以用这些$\alpha$来表示。

#### - SMO序列最小优化算法：

> - SMO算法的目的是求出一系列$\alpha$和$b$，一旦求出这些$\alpha$就可以很轻易的求出权重向量并得到分隔超平面。
> - SMO算法的基本原理是：在每次循环中选择一个$\alpha$对进行优化处理，只要这个$\alpha$对满足两个条件:
> - 1. 两个$\alpha$必须要在间隔边界之外
> - 2. 这两个$\alpha$还没有经过区间化处理或者不再边界上
